In [1]:
import pandas as pd 
import scsims
from sklearn.preprocessing import LabelEncoder
import sys

sys.path.append('../src')
from models.lib.lightning_train import DataModule, generate_trainer
from models.lib.neural import GeneClassifier

Let's do some simple preprocessing for training, and then test our library

In [2]:
# le = LabelEncoder()

# labels = pd.read_csv('../data/retina/retina_labels.tsv', sep='\t')
# labels = labels[labels['CellType'] != 'unannotated']
# labels['class_label'] = le.fit_transform(labels['CellType'])
# labels.index.name = 'cell'

# labels.to_csv('../data/retina/retina_labels_numeric.csv')

# pd.read_csv('../data/retina/retina_labels_numeric.csv')

In [3]:
datafiles = ['../data/retina/retina_T.csv']
labelfiles = ['../data/retina/retina_labels_numeric.csv']

module = DataModule(
    datafiles=['../data/retina/retina_T.csv'],
    labelfiles=['../data/retina/retina_labels_numeric.csv'],
    class_label='class_label',
    batch_size=4,
    num_workers=0,
    skip=3,
    shuffle=True,
    drop_last=True,
    index_col='cell',
)



Creating train/val/test DataLoaders...
Extension .csv not recognized,                 interpreting as .csv. To silence this warning, pass in explicit file types.
Done, continuing to training.


In [4]:
model = GeneClassifier(
    input_dim=module.num_features,
    output_dim=module.num_labels,
)

Model initialized. input_dim = 37475, output_dim = 13. Metrics are dict_keys(['accuracy', 'precision', 'recall']) and weighted_metrics = False


In [5]:
trainer, model, module = generate_trainer(
    datafiles=['../data/retina/retina_T.csv'],
    labelfiles=['../data/retina/retina_labels_numeric.csv'],
    class_label='class_label',
    batch_size=4,
    num_workers=0,
    skip=3,
    shuffle=True,
    drop_last=True,
    index_col='cell',
    weighted_metrics=False,
    normalize=True,
)

trainer.fit(model, datamodule=module)

Device is cpu
../data/retina/retina_T.csv exists, continuing...
../data/retina/retina_labels_numeric.csv exists, continuing...



GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Model initialized. input_dim = 37475, output_dim = 13. Metrics are dict_keys(['accuracy', 'precision', 'recall']) and weighted_metrics = False


In [ ]:
trainer.fit(model, datamodule=module)

Creating train/val/test DataLoaders...
Extension .csv not recognized,                 interpreting as .csv. To silence this warning, pass in explicit file types.
Done, continuing to training.


wandb: Currently logged in as: jlehrer1 (use `wandb login --relogin` to force relogin)



  | Name       | Type                 | Params
----------------------------------------------------
0 | base_model | TabNetGeneClassifier | 2.5 M 
----------------------------------------------------
2.5 M     Trainable params
0         Non-trainable params
2.5 M     Total params
9.916     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:453: UserWarning: Your `val_dataloader` has `shuffle=True`,it is strongly recommended that you turn this off for val/test/predict dataloaders.
  rank_zero_warn(
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of 

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]